# Lab 4: Multi-GPU DDP Training with Unified Tracking

## Overview
Master Distributed Data Parallel (DDP) training with simultaneous tracking across TensorBoard, MLflow, and WandB. Learn when to use DDP vs FSDP and how to integrate multiple monitoring tools.

## Learning Objectives
- Understand DDP architecture and use cases
- Configure multi-GPU training with PyTorch DDP
- Integrate TensorBoard, MLflow, and WandB simultaneously
- Compare DDP vs FSDP performance
- Implement production-ready experiment tracking

## Prerequisites
- Completed Labs 1-3
- MLflow server (optional)
- WandB account

**Estimated Time:** 60 minutes

## DDP vs FSDP: When to Use What?

### Distributed Data Parallel (DDP)
```
GPU 0: [Full Model] + [Batch 0]
GPU 1: [Full Model] + [Batch 1]
GPU 2: [Full Model] + [Batch 2]
GPU 3: [Full Model] + [Batch 3]
```

### Fully Sharded Data Parallel (FSDP)
```
GPU 0: [Model Shard 0] + [Batch 0]
GPU 1: [Model Shard 1] + [Batch 1]
GPU 2: [Model Shard 2] + [Batch 2]
GPU 3: [Model Shard 3] + [Batch 3]
```

### Decision Matrix

| Scenario | Use DDP | Use FSDP |
|----------|---------|----------|
| Model fits in single GPU | ✓ | ✗ |
| Model > GPU memory | ✗ | ✓ |
| Parameters < 1B | ✓ | ✗ |
| Parameters > 1B | ✗ | ✓ |
| Need max speed | ✓ | ✗ |
| Need max memory | ✗ | ✓ |

**This Lab Uses DDP** because SegResNet/SwinUNETR fit in single GPU memory.

## Step 1: Setup Environment

In [ ]:
%pip install sagemaker==2.200.0

In [ ]:
import sagemaker
from sagemaker.pytorch.estimator import PyTorch
from sagemaker import get_execution_role
import boto3

sagemaker_session = sagemaker.Session(boto3.Session(region_name='us-east-1'))
# sagemaker_session = sagemaker.Session()
role = get_execution_role()

region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()

print(f"Region: {region}")
print(f"Bucket: {bucket}")

## Step 2: Configure Tracking Tools

Enable all three tracking systems for comprehensive monitoring.

In [ ]:
import getpass

# TensorBoard (always enabled)
use_tensorboard = True

# MLflow (optional - requires server)
use_mlflow = True
mlflow_tracking_uri = ""  # e.g., "http://mlflow.example.com:5000"
# Create an MLflow tracking server in SageMaker and provide its URI here
# mlflow_tracking_uri = "arn:aws:sagemaker:us-east-1:xxxx"  # e.g., "http://mlflow-server:5000"
mlflow_experiment_name = "medical-seg-ddp-workshop"

# WandB (recommended)
use_wandb = False
wandb_api_key = getpass.getpass("Enter WandB API key (or press Enter to skip): ")
wandb_project = "medical-seg-ddp-workshop"

if not wandb_api_key:
    use_wandb = False

print("\n📊 Tracking Configuration:")
print(f"  TensorBoard: ✓ Enabled")
print(f"  MLflow: {'✓ Enabled' if use_mlflow else '✗ Disabled'}")
print(f"  WandB: {'✓ Enabled' if use_wandb else '✗ Disabled'}")

## Step 3: Configure Data Paths

In [ ]:
bucket = 'public-datasets-imaging-us-east-1'
data_path = f's3://{bucket}/segmentation_data/'
output_path = f's3://{bucket}/segmentation_data/output'

print(f"Training data: {data_path}")
print(f"Output path: {output_path}")

## Step 4: Experiment 1 - 2 GPU Training

Start with 2 GPUs to understand DDP basics.

In [ ]:
hyperparameters_2gpu = {
    "model_name": "SegResNet",
    "batch_size": 4,  # Per GPU
    "epochs": 15,
    "lr": 1e-4,
    "use_mlflow": str(use_mlflow),
    "use_wandb": str(use_wandb),
}

if use_mlflow:
    hyperparameters_2gpu["mlflow_tracking_uri"] = mlflow_tracking_uri
    hyperparameters_2gpu["mlflow_experiment_name"] = mlflow_experiment_name

if use_wandb:
    hyperparameters_2gpu["wandb_project"] = wandb_project
    hyperparameters_2gpu["wandb_api_key"] = wandb_api_key

estimator_2gpu = PyTorch(
    entry_point="train_ddp_all.py",
    source_dir="../code/training",
    role=role,
    instance_count=1,
    instance_type="ml.g4dn.12xlarge",  # 2 GPUs
    framework_version="2.1.0",
    py_version="py310",
    hyperparameters=hyperparameters_2gpu,
    output_path=output_path,
    base_job_name="ddp-2gpu-segresnet",
    keep_alive_period_in_seconds=1800,
    distribution={
        "pytorchddp": {
            "enabled": True
        },
    },
    sagemaker_session=sagemaker_session
)
print("✓ Experiment 1 configured: 2 GPU DDP")
print(f"  Effective batch size: {hyperparameters_2gpu['batch_size'] * 2}")

In [ ]:
# Launch training
estimator_2gpu.fit({"training": data_path}, wait=True, logs="All")

## Step 6: Analyze Training Metrics

Compare performance across tracking systems.

In [ ]:
import pandas as pd

# Get training job details
sm_client = boto3.client('sagemaker', region_name=region)

jobs = [
    estimator_2gpu.latest_training_job.name,
]

results = []
for job_name in jobs:
    job = sm_client.describe_training_job(TrainingJobName=job_name)
    results.append({
        "Job": job_name.split('-')[-3],
        "GPUs": 2 if '4xlarge' in job['ResourceConfig']['InstanceType'] else 4,
        "Model": job['HyperParameters']['model_name'],
        "Training Time (min)": job['TrainingTimeInSeconds'] / 60,
        "Billable Time (min)": job['BillableTimeInSeconds'] / 60,
        "Instance": job['ResourceConfig']['InstanceType'],
        "Status": job['TrainingJobStatus']
    })

df = pd.DataFrame(results)
print("\n📊 Training Job Comparison:")
print(df.to_string(index=False))

## Step 7: View TensorBoard Logs

In [ ]:
# Download TensorBoard logs from latest job
latest_job = estimator_2gpu.latest_training_job.name
model_data = estimator_2gpu.model_data

!aws s3 cp {model_data} ./model.tar.gz
!tar -xzf model.tar.gz

print("\n✓ TensorBoard logs downloaded")
print("\nLaunch TensorBoard:")
print("  tensorboard --logdir=./")
print("\nThen open: http://localhost:6006")

## Step 8: View WandB Dashboard

In [ ]:
if use_wandb:
    import wandb
    
    wandb.login(key=wandb_api_key)
    api = wandb.Api()
    
    runs = api.runs(f"{api.default_entity}/{wandb_project}")
    
    print(f"\n🎯 WandB Dashboard:")
    print(f"   https://wandb.ai/{api.default_entity}/{wandb_project}")
    print(f"\n📈 Found {len(runs)} runs")
    
    for run in runs[:5]:
        print(f"\n  Run: {run.name}")
        print(f"    Model: {run.config.get('model_name')}")
        print(f"    Best Dice: {run.summary.get('val/best_dice', 'N/A')}")
        print(f"    Status: {run.state}")
else:
    print("WandB not enabled. Set use_wandb=True to view dashboard.")

## Step 9: MLflow Tracking (Optional)

In [ ]:
if use_mlflow:
    import mlflow
    
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    # List experiments
    experiments = mlflow.search_experiments()
    
    print(f"\n🔬 MLflow Experiments:")
    for exp in experiments:
        if mlflow_experiment_name in exp.name:
            print(f"\n  Experiment: {exp.name}")
            print(f"    ID: {exp.experiment_id}")
            
            # Get runs
            runs = mlflow.search_runs(experiment_ids=[exp.experiment_id])
            print(f"    Runs: {len(runs)}")
            
            if len(runs) > 0:
                best_run = runs.loc[runs['metrics.best_dice'].idxmax()]
                print(f"\n  Best Run:")
                print(f"    Dice: {best_run['metrics.best_dice']:.4f}")
                print(f"    Model: {best_run['params.model_name']}")
else:
    print("MLflow not enabled. Set use_mlflow=True and configure server.")

## Step 10: Calculate Scaling Efficiency

In [ ]:
# Calculate speedup and efficiency
if len(results) >= 2:
    time_2gpu = results[0]['Training Time (min)']
    time_4gpu = results[1]['Training Time (min)']
    
    speedup = time_2gpu / time_4gpu
    efficiency = (speedup / 2) * 100  # 2x more GPUs
    
    print("\n⚡ Scaling Analysis:")
    print(f"  2 GPU Time: {time_2gpu:.1f} min")
    print(f"  4 GPU Time: {time_4gpu:.1f} min")
    print(f"  Speedup: {speedup:.2f}x")
    print(f"  Efficiency: {efficiency:.1f}%")
    print(f"\n  {'✓ Excellent' if efficiency > 85 else '⚠ Good' if efficiency > 70 else '✗ Poor'} scaling efficiency")

## Understanding DDP Architecture

### How DDP Works

1. **Initialization**
   - Each GPU gets full model copy
   - Process group established (NCCL)
   - Data distributed via DistributedSampler

2. **Forward Pass**
   - Each GPU processes its batch independently
   - No communication between GPUs

3. **Backward Pass**
   - Gradients computed locally
   - All-reduce operation synchronizes gradients
   - Each GPU has identical gradients

4. **Optimizer Step**
   - Each GPU updates its model copy
   - Models stay synchronized

### Communication Patterns

```
Forward:  GPU0 → GPU0  (no communication)
          GPU1 → GPU1
          GPU2 → GPU2
          GPU3 → GPU3

Backward: GPU0 ←→ GPU1 ←→ GPU2 ←→ GPU3  (all-reduce)
```

### Memory Usage

Per GPU:
- Model parameters: ~5GB (SwinUNETR)
- Gradients: ~5GB
- Optimizer states: ~10GB
- Activations: ~4GB (batch_size=2)
- **Total: ~24GB** (fits in A10G)

## Tracking System Comparison

### TensorBoard
✓ Built-in, no setup  
✓ Local visualization  
✓ Real-time updates  
✗ No cloud hosting  
✗ Limited collaboration  

**Best for:** Quick local experiments

### MLflow
✓ Model registry  
✓ Experiment comparison  
✓ Self-hosted control  
✗ Requires server setup  
✗ Limited real-time features  

**Best for:** Production ML pipelines

### WandB
✓ Cloud-hosted  
✓ Real-time collaboration  
✓ Advanced visualizations  
✓ Hyperparameter sweeps  
✗ External dependency  

**Best for:** Team collaboration and research

### Recommendation
Use **all three** for production:
- TensorBoard: Quick debugging
- MLflow: Model versioning
- WandB: Team collaboration

## Key Takeaways

✓ **DDP Advantages:**
- Faster than FSDP for models that fit in GPU
- Simpler implementation
- Better scaling efficiency (85-95%)
- Lower communication overhead

✓ **Multi-Tracking Benefits:**
- TensorBoard for immediate feedback
- MLflow for production tracking
- WandB for team collaboration
- Redundancy and flexibility

✓ **Best Practices:**
- Use DDP when model fits in single GPU
- Batch size per GPU: 2-4 for medical imaging
- Monitor GPU utilization (aim for >80%)
- Enable gradient accumulation for larger effective batch sizes

## Cost Analysis

| Configuration | Instance | Time | Cost/Hour | Total Cost |
|---------------|----------|------|-----------|------------|
| 2 GPU DDP | ml.g5.4xlarge | 25 min | $2.83 | $1.18 |
| 4 GPU DDP | ml.g5.12xlarge | 15 min | $7.09 | $1.77 |
| **Difference** | | **-40%** | **+150%** | **+50%** |

**Insight:** 4 GPUs cost 50% more but train 40% faster - good for production.

## Next Steps

- Implement gradient accumulation for larger batch sizes
- Try mixed precision training (FP16)
- Set up automated hyperparameter tuning
- Deploy best model to SageMaker Endpoint
- Create CI/CD pipeline with tracking integration

## Additional Resources

- [PyTorch DDP Tutorial](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)
- [SageMaker Distributed Training](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html)
- [NCCL Performance Tuning](https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/usage/tuning.html)